In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps

import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

In [2]:
def remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    "Remove apertures of off-momentum BPMs which give flanges as bottlenecks"
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    aper_to_remove = [f'{name}{suffix}' for name in offmom_bpms for suffix in ('.a_aper', '.b_aper')]
    line.remove(aper_to_remove)

# Error lines

In [13]:
start_chroma = -1.5
end_chroma = 1.5
chromas_norm = np.round(
    np.linspace(start_chroma, end_chroma,
                int((end_chroma - start_chroma)/0.1) + 1),
    1
)
qx = 20.13
qy = 20.18

In [17]:
for c in chromas_norm:
    print(f'Treating normalised chroma {c}')
    # Preparing line
    line = xt.Line.from_json('../../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
    env = line.env
    cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

    for name in cavity_names:
        line[name].frequency = 200e6
        line[name].lag = 180
    line['acl.31735'].voltage = 0 #setting 800 cav to 0V
    line['actcse.31632'].voltage = 3.0e6

    tw = line.twiss()
    remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)

    # Installing errors
    tte = env.elements.get_table()
    mask_rbends = tte.element_type == 'RBend'
    mask_quads = tte.element_type == 'Quadrupole'
    mask_sextupoles = tte.element_type == 'Sextupole'

    mba = tte.rows[mask_rbends].rows['mba.*'].name
    mbb = tte.rows[mask_rbends].rows['mbb.*'].name
    qf = tte.rows[mask_quads].rows['qf.*'].name
    qd = tte.rows[mask_quads].rows['qd.*'].name
    lsf = tte.rows[mask_sextupoles].rows['lsf.*'].name
    lsd = tte.rows[mask_sextupoles].rows['lsd.*'].name

    env.vars['qph_setvalue'] = 0.0
    env.vars['qpv_setvalue'] = 0.0

    # Set the strengths according to Hannes' measurements
    for nn in mba:
        env[nn].knl = np.array([0., 0., 2.12e-3, 0., -5.74, 0.])

    for nn in mbb:
        env[nn].knl = np.array([0., 0., -3.19e-3, 0., -5.10, 0.])

    for nn in qf:
        env[nn].knl = np.array([0., 0., 0., 0.75e-1, 0., -0.87e3])

    for nn in qd:
        env[nn].knl = np.array([0., 0., 0., -2.03e-1, 0., 2.04e3])

    env.vars['qph_setvalue'] = c
    env.vars['qpv_setvalue'] = c
    opt = line.match(
        method='6d', # <- passed to twiss
        vary=[
            xt.VaryList(['kqf0', 'kqd0'], step=1e-8, tag='quad'),
            xt.VaryList(['qph_setvalue', 'qpv_setvalue'], step=1e-4, tag='sext'),
        ],
        targets = [
            xt.TargetSet(qx=qx, qy=qy, tol=1e-6, tag='tune'),
            xt.TargetSet(dqx=c*qx, dqy=c*qy, tol=1e-2, tag='chrom'),
        ])

    line.to_json(f'lines_simulations/sps_errors_chroma_{c}.json')
    print(' ')

Treating normalised chroma -1.5


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 13                                
Matching: model call n. 13 penalty = 1.4127e-05              
Optimize - end penalty:  1.41274e-05                            
Treating normalised chroma -1.4


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 13.01                             
Matching: model call n. 13 penalty = 2.1795e-05              
Optimize - end penalty:  2.17947e-05                            
Treating normalised chroma -1.3


...

Done loading line from dict.           
                                             
Optimize - start penalty: 13.11                             
Matching: model call n. 13 penalty = 4.9918e-06              
Optimize - end penalty:  4.9918e-06                            
Treating normalised chroma 1.4


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 13.11                             
Matching: model call n. 13 penalty = 1.5923e-06              
Optimize - end penalty:  1.59225e-06                            
Treating normalised chroma 1.5


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 13.12                             
Matching: model call n. 13 penalty = 2.2103e-06              
Optimize - end penalty:  2.21034e-06                            


# Linear lines

In [18]:
for c in chromas_norm:
    print(f'Treating normalised chroma {c}')
    # Preparing line
    line = xt.Line.from_json('../../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
    env = line.env
    cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

    for name in cavity_names:
        line[name].frequency = 200e6
        line[name].lag = 180
    line['acl.31735'].voltage = 0 #setting 800 cav to 0V
    line['actcse.31632'].voltage = 3.0e6

    tw = line.twiss()
    remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)

    env.vars['qph_setvalue'] = c
    env.vars['qpv_setvalue'] = c
    opt = line.match(
        method='6d', # <- passed to twiss
        vary=[
            xt.VaryList(['kqf0', 'kqd0'], step=1e-8, tag='quad'),
            xt.VaryList(['qph_setvalue', 'qpv_setvalue'], step=1e-4, tag='sext'),
        ],
        targets = [
            xt.TargetSet(qx=qx, qy=qy, tol=1e-6, tag='tune'),
            xt.TargetSet(dqx=c*qx, dqy=c*qy, tol=1e-2, tag='chrom'),
        ])

    line.to_json(f'lines_simulations/sps_chroma_{c}.json')
    print(' ')

Treating normalised chroma -1.5


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 0.1529                            
Matching: model call n. 7 penalty = 4.5329e-04              
Optimize - end penalty:  0.000453292                            
 
Treating normalised chroma -1.4


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 0.152                             
Matching: model call n. 7 penalty = 2.5990e-04              
Optimize - end penalty:  0.000259895                            
 
Treating normalised chroma -1.3


...

Done loading line from dict.           
                                             
Optimize - start penalty: 0.2024                            
Matching: model call n. 7 penalty = 4.1087e-04              
Optimize - end penalty:  0.000410872                            
 
Treating normalised chroma 1.4


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 0.2063                            
Matching: model call n. 7 penalty = 7.3497e-04              
Optimize - end penalty:  0.000734972                            
 
Treating normalised chroma 1.5


Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
                                             
Optimize - start penalty: 0.2104                            
Matching: model call n. 7 penalty = 1.2998e-03              
Optimize - end penalty:  0.00129975                            
 
